In [9]:
# https://www.tensorflow.org/tutorials/keras/regression

from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import os



1.14.0


In [10]:
# Import helpers
import import_ipynb
from common import buildDataset
from common import scaleVolume
from common import filterBad
from common import splitDataset
from common import dropColumns
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractNNWeights

In [11]:
only_use_strong = 54

dataset, columns = buildDataset()
dataset = filterBad(dataset, only_use_strong)
dataset = scaleVolume(dataset, 4096)
dataset, columns = dropColumns(dataset, ['fileId'])
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.1)
train_stats = train_dataset.describe().transpose()

# FCNN-specific
normed_train_data = normDataset(train_dataset, train_stats)
normed_test_data = normDataset(test_dataset, train_stats)

# print(train_dataset)
# print(normed_train_data)

Rejected 6196746 points (56.847699%)


In [12]:
print(len(train_dataset))
print(len(test_dataset))

3763091
940773


In [13]:
def buildNN(hLayers, actFuncs):
    nn = []
    for i in range(len(hLayers)):
        if (i == 0):
            shape = [len(train_dataset.keys())]
        else:
            shape = [hLayers[i-1]]
        nn.append(layers.Dense(hLayers[i], activation=actFuncs[i], input_shape = shape))
    nn.append(layers.Dense(1))
    model = keras.Sequential(nn)

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model

In [14]:
def trainNN(model, validation_split, epochs):
    history = model.fit(
      normed_train_data, train_labels,
      epochs=epochs, validation_split = validation_split, verbose=1,
      callbacks=[EarlyStopping(monitor='val_loss', patience=2)])
    #     callbacks = callbacks_list)

    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist.tail()
    return model


In [15]:
def trainAndBuild(hLayers, actFuncs, validation_split, epochs):
    model = buildNN(hLayers, actFuncs)
    model.summary()
    model = trainNN(model, validation_split, epochs)
    return model
    

# Model playground

In [16]:
show_perf = True
load_prior_model = False
train_model = True
strong_data_sfx = "_strong" if (only_use_strong > 0) else ""

# hDims = [xxx,yyy]
# actFuncs = ['relu', 'relu']
# NN_xxx_yyy_zzz = buildNN(hDims, actFuncs)
# if (start_from_checkpoint): NN_xxx_yyy_zzz.load_weights('./NN
# import pandas as pd_xxx_yyy_zzz') #load
# # Either train or load weights
# if (train_model): NN_xxx_yyy_zzz = trainNN(NN_xxx_yyy_zzz, 0.15, 15) #train
# else: NN_xxx_yyy_zzz.load_weights('./NN_xxx_yyy_zzz') #load
# # Save weights
# NN_xxx_yyy_zzz.save_weights('./NN_xxx_yyy_zzz')
# preds = NN_xxx_yyy_zzz.predict(normed_test_data)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractNNWeights(NN_xxx_yyy_zzz, 'NN_xxx_yyy_zzz', hDims, columns, train_stats)
# print('--------------------------------')

# hDims = [64]
# actFuncs = ['relu']
# NN_64_RELU = buildNN(hDims, actFuncs)
# if (load_prior_model): NN_64_RELU.load_weights('./NN_64_RELU' + strong_data_sfx) #load
# # Either train or load weights
# if (train_model): 
#     NN_64_RELU = trainNN(NN_64_RELU, 0.05, 15) #train
# #     NN_64_RELU.save_weights('./NN_64_RELU' + strong_data_sfx)
# preds = NN_64_RELU.predict(normed_test_data)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractNNWeights(NN_64_RELU, 'NN_64_RELU' + strong_data_sfx, hDims, columns, train_stats)
# print('--------------------------------')

# hDims = [64]
# actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.25)]
# NN_64_Leaky = buildNN(hDims, actFuncs)
# # Either train or load weights
# if (train_model): NN_64_Leaky = trainNN(NN_64_Leaky, 0.05, 10) #train
# else: NN_64_Leaky.load_weights('./NN_64_Leaky' + strong_data_sfx) #load
# # Save weights
# NN_64_Leaky.save_weights('./NN_64_Leaky' + strong_data_sfx)
# preds = NN_64_Leaky.predict(normed_test_data)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractNNWeights(NN_64_Leaky, 'NN_64_Leaky' + strong_data_sfx, hDims, columns, train_stats)
# print('--------------------------------')


# hDims = [32,32]
# actFuncs = ['relu','relu']
# NN_32_32_RELU = buildNN(hDims, actFuncs)
# # Either train or load weights
# if (load_prior_model): NN_32_32_Leaky.load_weights('./NN_32_32_Leaky' + strong_data_sfx) #load
# if (train_model): 
#     NN_32_32_Leaky = trainNN(NN_32_32_Leaky, 0.05, 1) #train
#     NN_32_32_Leaky.save_weights('./NN_32_32_Leaky' + strong_data_sfx)
# preds = NN_32_32_RELU.predict(normed_test_data)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractNNWeights(NN_32_32_RELU, 'NN_32_32_RELU' + strong_data_sfx, hDims, columns, train_stats)
# print('--------------------------------')

# hDims = [32,32]
# actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
# NN_32_32_Leaky = buildNN(hDims, actFuncs)
# # Either train or load weights
# if (train_model): NN_32_32_Leaky = trainNN(NN_32_32_Leaky, 0.05, 10) #train
# else: NN_32_32_Leaky.load_weights('./NN_32_32_Leaky' + strong_data_sfx) #load
# # Save weights
# NN_32_32_Leaky.save_weights('./NN_32_32_Leaky' + strong_data_sfx)
# preds = NN_32_32_Leaky.predict(normed_test_data)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractNNWeights(NN_32_32_Leaky, 'NN_32_32_Leaky' + strong_data_sfx, hDims, columns, train_stats)
# print('--------------------------------')

# hDims = [16,16]
# actFuncs = ['relu', 'relu']
# NN_16_16_RELU = buildNN(hDims, actFuncs)
# # Either train or load weights
# if (train_model): NN_16_16_RELU = trainNN(NN_16_16_RELU, 0.05, 15) #train
# else: NN_16_16_RELU.load_weights('./NN_16_16_RELU' + strong_data_sfx) #load
# # Save weights
# NN_16_16_RELU.save_weights('./NN_16_16_RELU' + strong_data_sfx)
# preds = NN_16_16_RELU.predict(normed_test_data)
# if (show_perf): evaluatePerf(test_labels, preds)
# print('--------------------------------')
# extractNNWeights(NN_16_16_RELU, 'NN_16_16_RELU' + strong_data_sfx, hDims, columns, train_stats)
# print('--------------------------------')

hDims = [16,16]
actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
NN_16_16_Leaky = buildNN(hDims, actFuncs)
# Either train or load weights
if (train_model): NN_16_16_Leaky = trainNN(NN_16_16_Leaky, 0.05, 10) #train
else: NN_16_16_Leaky.load_weights('./NN_16_16_Leaky' + strong_data_sfx) #load
# Save weights
NN_16_16_Leaky.save_weights('./NN_16_16_Leaky' + strong_data_sfx)
preds = NN_16_16_Leaky.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_16_16_Leaky, 'NN_16_16_Leaky' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')



hDims = [8,8]
actFuncs = ['relu', 'relu']
NN_8_8_RELU = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_8_8_RELU.load_weights('./NN_8_8_RELU' + strong_data_sfx) #load
if (train_model): 
    NN_8_8_RELU = trainNN(NN_8_8_RELU, 0.05, 8) #train
    NN_8_8_RELU.save_weights('./NN_8_8_RELU' + strong_data_sfx)
# Save weights
preds = NN_8_8_RELU.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_8_8_RELU, 'NN_8_8_RELU' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')

hDims = [4,4]
actFuncs = ['relu', 'relu']
NN_4_4_RELU = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_4_4_RELU.load_weights('./NN_4_4_RELU' + strong_data_sfx) #load
if (train_model): 
    NN_4_4_RELU = trainNN(NN_4_4_RELU, 0.05, 40) #train
    NN_4_4_RELU.save_weights('./NN_4_4_RELU' + strong_data_sfx)
preds = NN_4_4_RELU.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_4_4_RELU, 'NN_4_4_RELU' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')


hDims = [16,4]
actFuncs = ['relu', 'relu']
NN_16_4_RELU = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_16_4_RELU.load_weights('./NN_16_4_RELU' + strong_data_sfx) #load
if (train_model): 
    NN_16_4_RELU = trainNN(NN_16_4_RELU, 0.05, 8) #train
    NN_16_4_RELU.save_weights('./NN_16_4_RELU' + strong_data_sfx)
preds = NN_16_4_RELU.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_16_4_RELU, 'NN_16_4_RELU' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')


hDims = [8,4]
actFuncs = ['relu', 'relu']
NN_8_4_RELU = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_8_4_RELU.load_weights('./NN_8_4_RELU' + strong_data_sfx) #load
if (train_model): 
    NN_8_4_RELU = trainNN(NN_8_4_RELU, 0.05, 40) #train
    NN_8_4_RELU.save_weights('./NN_8_4_RELU' + strong_data_sfx)
preds = NN_8_4_RELU.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_8_4_RELU, 'NN_8_4_RELU' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')


hDims = [8,8]
actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
NN_8_8_Leaky = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_8_8_Leaky.load_weights('./NN_8_8_Leaky' + strong_data_sfx) #load
if (train_model): 
    NN_8_8_Leaky = trainNN(NN_8_8_Leaky, 0.05, 40) #train
    NN_8_8_Leaky.save_weights('./NN_8_8_Leaky' + strong_data_sfx)
# Save weights
preds = NN_8_8_Leaky.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_8_8_Leaky, 'NN_8_8_Leaky' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')

hDims = [4,4]
actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
NN_4_4_Leaky = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_4_4_Leaky.load_weights('./NN_4_4_Leaky' + strong_data_sfx) #load
if (train_model): 
    NN_4_4_Leaky = trainNN(NN_4_4_Leaky, 0.05, 40) #train
    NN_4_4_Leaky.save_weights('./NN_4_4_Leaky' + strong_data_sfx)
preds = NN_4_4_Leaky.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_4_4_Leaky, 'NN_4_4_Leaky' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')


hDims = [16,4]
actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
NN_16_4_Leaky = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_16_4_Leaky.load_weights('./NN_16_4_Leaky' + strong_data_sfx) #load
if (train_model): 
    NN_16_4_Leaky = trainNN(NN_16_4_Leaky, 0.05, 40) #train
    NN_16_4_Leaky.save_weights('./NN_16_4_Leaky' + strong_data_sfx)
preds = NN_16_4_Leaky.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_16_4_Leaky, 'NN_16_4_Leaky' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')


hDims = [8,4]
actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
NN_8_4_Leaky = buildNN(hDims, actFuncs)
# Either train or load weights
if (load_prior_model): NN_8_4_Leaky.load_weights('./NN_8_4_Leaky' + strong_data_sfx) #load
if (train_model): 
    NN_8_4_Leaky = trainNN(NN_8_4_Leaky, 0.05, 40) #train
    NN_8_4_Leaky.save_weights('./NN_8_4_Leaky' + strong_data_sfx)
preds = NN_8_4_Leaky.predict(normed_test_data)
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractNNWeights(NN_8_4_Leaky, 'NN_8_4_Leaky' + strong_data_sfx, hDims, columns, train_stats)
print('--------------------------------')


Train on 3574936 samples, validate on 188155 samples
Epoch 1/10
3574936/3574936 [==============================] - 375s 105us/sample - loss: 11577.0482 - mean_absolute_error: 23.0494 - mean_squared_error: 11576.1709 - val_loss: 1084.5155 - val_mean_absolute_error: 10.9644 - val_mean_squared_error: 1084.5150
Epoch 2/10
3574936/3574936 [==============================] - 375s 105us/sample - loss: 755.0740 - mean_absolute_error: 10.5812 - mean_squared_error: 755.0755 - val_loss: 612.6657 - val_mean_absolute_error: 10.0866 - val_mean_squared_error: 612.6664
Epoch 3/10
3574936/3574936 [==============================] - 376s 105us/sample - loss: 575.7259 - mean_absolute_error: 10.2663 - mean_squared_error: 575.7250 - val_loss: 534.9476 - val_mean_absolute_error: 10.2102 - val_mean_squared_error: 534.9473
Epoch 4/10
3574936/3574936 [==============================] - 376s 105us/sample - loss: 580.9171 - mean_absolute_error: 10.2185 - mean_squared_error: 580.9175 - val_loss: 619.2899 - val_mean_

Train on 3574936 samples, validate on 188155 samples
Epoch 1/40
3574936/3574936 [==============================] - 385s 108us/sample - loss: 166921.8028 - mean_absolute_error: 145.9279 - mean_squared_error: 166778.8125 - val_loss: 923.0670 - val_mean_absolute_error: 17.2487 - val_mean_squared_error: 923.0654
Epoch 2/40
3574936/3574936 [==============================] - 382s 107us/sample - loss: 946.9818 - mean_absolute_error: 16.1383 - mean_squared_error: 946.9800 - val_loss: 887.5126 - val_mean_absolute_error: 15.7700 - val_mean_squared_error: 887.5123
Epoch 3/40
3574936/3574936 [==============================] - 379s 106us/sample - loss: 899.5041 - mean_absolute_error: 15.9554 - mean_squared_error: 899.5049 - val_loss: 850.9267 - val_mean_absolute_error: 15.8481 - val_mean_squared_error: 850.9271
Epoch 4/40
3574936/3574936 [==============================] - 377s 106us/sample - loss: 881.5131 - mean_absolute_error: 15.9459 - mean_squared_error: 881.5168 - val_loss: 847.9839 - val_mean

Train on 3574936 samples, validate on 188155 samples
Epoch 1/40
3574936/3574936 [==============================] - 387s 108us/sample - loss: 29757.9760 - mean_absolute_error: 41.9600 - mean_squared_error: 29763.4766 - val_loss: 851.1909 - val_mean_absolute_error: 13.3075 - val_mean_squared_error: 851.1910
Epoch 2/40
3574936/3574936 [==============================] - 384s 107us/sample - loss: 940.1567 - mean_absolute_error: 12.9076 - mean_squared_error: 940.1654 - val_loss: 1242.4164 - val_mean_absolute_error: 12.5245 - val_mean_squared_error: 1242.4175
Epoch 3/40
3574936/3574936 [==============================] - 379s 106us/sample - loss: 1266.1980 - mean_absolute_error: 12.4214 - mean_squared_error: 1266.1958 - val_loss: 1231.7218 - val_mean_absolute_error: 12.1045 - val_mean_squared_error: 1231.7203
MSE, MAE, r2: 1366.019206,12.162000,0.998696
--------------------------------
  def NN_8_4_RELU_strong_mean[I,T](toI: scala.Double => I, toT: scala.Double => T) = (toI(26),toI(253.172799)

3574936/3574936 [==============================] - 398s 111us/sample - loss: 20002.8880 - mean_absolute_error: 31.4256 - mean_squared_error: 20011.3281 - val_loss: 1071.0680 - val_mean_absolute_error: 11.4726 - val_mean_squared_error: 1071.0670
Epoch 2/40
3574936/3574936 [==============================] - 393s 110us/sample - loss: 958.5761 - mean_absolute_error: 11.2954 - mean_squared_error: 958.5789 - val_loss: 780.3355 - val_mean_absolute_error: 11.0087 - val_mean_squared_error: 780.3344
Epoch 3/40
3574936/3574936 [==============================] - 376s 105us/sample - loss: 677.9906 - mean_absolute_error: 10.9380 - mean_squared_error: 677.9816 - val_loss: 610.3681 - val_mean_absolute_error: 10.6758 - val_mean_squared_error: 610.3690
Epoch 4/40
3574936/3574936 [==============================] - 280s 78us/sample - loss: 598.2742 - mean_absolute_error: 10.6620 - mean_squared_error: 598.2756 - val_loss: 637.7739 - val_mean_absolute_error: 10.8216 - val_mean_squared_error: 637.7729
Epoch 

In [26]:
df2 = pd.DataFrame({
    "row":[50],
    "rising_idx":[157],
    "falling_idx":[341],
    "volume":[1613648/4096],
    "rising_weight":[35.2498],
    "falling_weight":[-1028.1037], 
    "first_val":[0],
    "last_val":[0]})
# df2 = pd.DataFrame({
#     "row":[53],
#     "rising_idx":[258.314512],
#     "falling_idx":[436.267106],
#     "volume":[1613648/4096],
#     "rising_weight":[13208.708076],
#     "falling_weight":[-3395.294660], 
#     "first_val":[1308.543584],
#     "last_val":[7.874365]})

evaluateCustom(NN_64_RELU, normDataset(df2, train_stats), 509.093)
# evaluateCustom(NN_64_Leaky, normDataset(df2, train_stats), 509.093)
# evaluateCustom(NN_32_32_RELU, normDataset(df2, train_stats), 509.093)
# evaluateCustom(NN_32_32_Leaky, normDataset(df2, train_stats), 509.093)

[[417.0136]]
True label = 509.093000


In [ ]:
print('hi')